# AORC HMS Execution and Results

This notebook demonstrates how to execute HEC-HMS simulations using AORC gridded precipitation and analyze the results.

**Workflow Overview**:
1. Initialize HMS project with AORC grids
2. Execute HMS simulation
3. Extract peak flows and time series
4. Visualize results

**Prerequisites**:
- Complete **14a_aorc_download.ipynb** and **14b_aorc_grid_setup.ipynb** first
- HEC-HMS 4.x installation with example projects

**Series Navigation**:
- **14a**: AORC download and storm catalog
- **14b**: HMS grid definition and HRAP mapping
- **14c** (this notebook): HMS execution and results analysis

In [ ]:
# pip install hms-commander[all]

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

## Setup and Imports

In [ ]:
from pathlib import Path
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# HMS Commander imports
from hms_commander import (
    init_hms_project, HmsCmdr, HmsResults, HmsBasin, 
    HmsExamples, hms
)

print(f"Workflow started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Extract Example Project

The `tenk` example project is the only HMS example with gridded precipitation support. We'll use it as a template for demonstration.

In [ ]:
# Extract the tenk example project (required for HMS operations)
print("Extracting tenk example project...")
try:
    tenk_path = HmsExamples.extract_project("tenk")
    print(f"  Extracted to: {tenk_path}")
    TENK_AVAILABLE = True
except Exception as e:
    print(f"  Warning: Could not extract tenk project: {e}")
    print("  Please ensure HEC-HMS is installed with example projects.")
    TENK_AVAILABLE = False
    tenk_path = None

---

## Initialize HMS Project

In [ ]:
if TENK_AVAILABLE:
    # Initialize the HMS project
    project_dir = tenk_path
    hms_project = init_hms_project(project_dir)

    print("HMS Project Initialized:")
    print(f"  Project: {hms_project.project_name}")
    print(f"  Directory: {hms_project.project_folder}")

    # Show available runs
    print(f"\nAvailable Runs ({len(hms_project.run_df)}):")
    for _, row in hms_project.run_df.iterrows():
        print(f"  - {row['name']}")
else:
    print("Skipping: tenk project not available")

In [ ]:
if TENK_AVAILABLE:
    # Show basin model subbasins
    print("Basin Model Subbasins:")
    print(hms_project.subbasin_df[['name', 'area', 'downstream']].head(10))
else:
    print("Skipping: tenk project not available")

---

## HMS Basin Model Geometry

Visualize the HMS basin model structure showing subbasins, junctions, and reaches.

In [ ]:
if TENK_AVAILABLE:
    try:
        import matplotlib.pyplot as plt
        import numpy as np

        basin_files = list(tenk_path.glob('*.basin'))
        if basin_files:
            basin_file = str(basin_files[0])
            print(f"Using basin file: {basin_file}")

            subbasins = HmsBasin.get_subbasins(basin_file)
            junctions = HmsBasin.get_junctions(basin_file)
            reaches = HmsBasin.get_reaches(basin_file)

            print(f"\nBasin Model Elements:")
            print(f"  Subbasins: {len(subbasins)}")
            print(f"  Junctions: {len(junctions)}")
            print(f"  Reaches:   {len(reaches)}")

            fig, ax = plt.subplots(figsize=(12, 10))

            # Plot subbasins as circles
            for idx, sub in subbasins.iterrows():
                x, y = float(sub['canvas_x']), float(sub['canvas_y'])
                size = np.sqrt(float(sub['area'])) * 3
                circle = plt.Circle((x, y), size, color='lightblue', ec='blue', linewidth=2, alpha=0.7)
                ax.add_patch(circle)
                ax.annotate(f"{sub['name']}\n({float(sub['area']):.0f} sq mi)",
                            (x, y), ha='center', va='center', fontsize=8, fontweight='bold')

            # Plot junctions as diamonds
            for idx, junc in junctions.iterrows():
                x, y = float(junc['canvas_x']), float(junc['canvas_y'])
                diamond = plt.matplotlib.patches.RegularPolygon(
                    (x, y), numVertices=4, radius=8,
                    orientation=np.pi/4, color='gold', ec='darkorange', linewidth=2
                )
                ax.add_patch(diamond)
                ax.annotate(junc['name'], (x, y+12), ha='center', fontsize=8)

            ax.set_xlim(-50, 550)
            ax.set_ylim(0, 550)
            ax.set_aspect('equal')
            ax.set_title('HMS Basin Model Schematic', fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        else:
            print("No basin files found")

    except ImportError:
        print("Matplotlib not available - skipping visualization")
else:
    print("Skipping: tenk project not available")

---

## Execute HMS Simulation

Run the HMS simulation using the gridded precipitation data.

In [ ]:
if TENK_AVAILABLE:
    # Execute HMS - run the first available run
    run_name = hms_project.run_df.iloc[0]['name']
    print(f"Executing HMS run: {run_name}")
    print("=" * 50)

    success = HmsCmdr.compute_run(run_name)

    if success:
        print(f"\nRun completed successfully!")
    else:
        print(f"\nRun failed or completed with warnings")
else:
    print("Skipping: tenk project not available")

---

## Analyze Results - Peak Flows

Extract and display peak flow results from the HMS simulation.

In [ ]:
if TENK_AVAILABLE:
    # Find the DSS results file
    dss_results = list(tenk_path.glob("**/*.dss"))
    print(f"Found {len(dss_results)} DSS files:")
    for dss in dss_results:
        print(f"  {dss.name}: {dss.stat().st_size / 1024:.1f} KB")

    # The results DSS is named after the run (with underscores replacing spaces)
    results_dss_list = [d for d in dss_results if "hrap" not in d.name.lower() and "tenk" not in d.name.lower()]
    if results_dss_list:
        results_dss = results_dss_list[0]
        print(f"\nUsing results DSS: {results_dss.name}")
    else:
        results_dss = None
        print("No results DSS file found")
else:
    print("Skipping: tenk project not available")

In [ ]:
if TENK_AVAILABLE and results_dss:
    # Extract peak flows from the results DSS file
    print(f"Analyzing results from: {results_dss.name}")

    try:
        # Get DSS catalog to see what's available
        from ras_commander.dss import RasDss
        catalog = RasDss.get_catalog(str(results_dss))
        
        # Filter for FLOW records
        flow_records = [p for p in catalog if 'FLOW' in p.upper()]
        print(f"\nFlow records found: {len(flow_records)}")
        
        # Get peak flows for all elements
        peaks = HmsResults.get_peak_flows(str(results_dss))
        
        if len(peaks) > 0:
            print(f"\nPeak Flow Summary ({len(peaks)} elements):")
            print("=" * 70)
            print(f"{'Element':<30} {'Peak Flow (cfs)':>15} {'Time to Peak':>20}")
            print("-" * 70)
            
            for element, row in peaks.iterrows():
                peak_flow = row.get('Peak Flow (cfs)', row.get('peak_flow', 'N/A'))
                time_peak = row.get('Time to Peak', row.get('time_to_peak', 'N/A'))
                if isinstance(peak_flow, (int, float)):
                    print(f"{element:<30} {peak_flow:>15.1f} {str(time_peak):>20}")
                else:
                    print(f"{element:<30} {str(peak_flow):>15} {str(time_peak):>20}")
        else:
            print("No peak flows extracted")
            
    except Exception as e:
        print(f"Error extracting peak flows: {e}")
else:
    print("Skipping: results not available")

### Per-Element Peak Flow Analysis

In [ ]:
if TENK_AVAILABLE and results_dss:
    try:
        from ras_commander.dss import RasDss

        catalog = RasDss.get_catalog(str(results_dss))
        
        flow_records = [p for p in catalog if '/FLOW/' in p.upper()
                        and 'OBSERVED' not in p.upper()
                        and 'CUMULATIVE' not in p.upper()
                        and 'RESIDUAL' not in p.upper()]

        # Extract peak flows for each element
        element_peaks = {}

        for pathname in flow_records:
            parts = pathname.split('/')
            if len(parts) >= 3:
                element_name = parts[2]

                try:
                    ts = RasDss.read_timeseries(str(results_dss), pathname)
                    if ts is not None and len(ts) > 0:
                        vals = ts.values.flatten() if ts.values.ndim > 1 else ts.values
                        peak_val = float(vals.max())
                        peak_idx = int(vals.argmax())
                        peak_time = ts.index[peak_idx]

                        if element_name not in element_peaks:
                            element_peaks[element_name] = []
                        element_peaks[element_name].append({
                            'pathname': pathname,
                            'peak_flow': peak_val,
                            'peak_time': peak_time,
                            'parameter': parts[3] if len(parts) > 3 else 'FLOW'
                        })
                except Exception:
                    pass

        print(f"Elements with peak flows: {len(element_peaks)}")
        for elem, peaks_list in element_peaks.items():
            max_peak = max(p['peak_flow'] for p in peaks_list)
            print(f"  {elem}: {len(peaks_list)} records, max peak: {max_peak:.0f} cfs")
            
    except Exception as e:
        print(f"Error analyzing elements: {e}")
else:
    print("Skipping: results not available")

### Peak Flow Summary Chart

In [ ]:
if TENK_AVAILABLE and 'element_peaks' in dir() and len(element_peaks) > 0:
    try:
        import matplotlib.pyplot as plt

        # Get max peak for each element
        all_elements = {}
        for elem, peaks_list in element_peaks.items():
            max_peak = max(peaks_list, key=lambda x: x['peak_flow'])
            all_elements[elem] = max_peak['peak_flow']

        sorted_elements = sorted(all_elements.items(), key=lambda x: x[1], reverse=True)
        names = [x[0] for x in sorted_elements]
        values = [x[1] for x in sorted_elements]

        fig, ax = plt.subplots(figsize=(12, 8))
        bars = ax.barh(range(len(names)), values, color='steelblue', alpha=0.8)
        ax.set_yticks(range(len(names)))
        ax.set_yticklabels(names)
        ax.set_xlabel('Peak Flow (cfs)')
        ax.set_title('Peak Flows by Element', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')

        # Add value labels
        for i, (name, val) in enumerate(sorted_elements):
            ax.text(val + max(values)*0.01, i, f'{val:,.0f}', va='center', fontsize=9)

        plt.tight_layout()
        plt.show()

    except ImportError:
        print("Matplotlib not available - skipping visualization")
else:
    print("Skipping: no element peak data available")

### Outlet Hydrograph

In [ ]:
if TENK_AVAILABLE and results_dss:
    try:
        import matplotlib.pyplot as plt
        from ras_commander.dss import RasDss
        
        # Get DSS catalog and find FLOW records
        catalog = RasDss.get_catalog(str(results_dss))
        flow_records = [p for p in catalog if 'FLOW' in p.upper() and 'OBSERVED' not in p.upper()]
        
        if flow_records:
            # Use first flow record
            pathname = flow_records[0]
            print(f"Plotting hydrograph for: {pathname}")
            
            ts = RasDss.read_timeseries(str(results_dss), pathname)
            
            if ts is not None and len(ts) > 0:
                fig, ax = plt.subplots(figsize=(12, 6))
                
                vals = ts.values.flatten() if ts.values.ndim > 1 else ts.values
                
                ax.plot(ts.index, vals, 'b-', linewidth=1.5)
                ax.fill_between(ts.index, vals, alpha=0.3)
                ax.set_xlabel('Time')
                ax.set_ylabel('Flow (cfs)')
                
                # Extract element name from pathname
                parts = pathname.split('/')
                element_name = parts[2] if len(parts) >= 3 else 'Outlet'
                
                # Mark peak
                peak_idx = vals.argmax()
                peak_val = vals.max()
                peak_time = ts.index[peak_idx]
                ax.axhline(y=peak_val, color='r', linestyle='--', alpha=0.5)
                ax.scatter([peak_time], [peak_val], color='red', s=100, zorder=5)
                ax.annotate(f'Peak: {peak_val:,.0f} cfs', 
                           xy=(peak_time, peak_val), xytext=(10, 10),
                           textcoords='offset points', fontsize=10,
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                
                ax.set_title(f'{element_name} Hydrograph', fontsize=14, fontweight='bold')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            else:
                print("No data in timeseries")
        else:
            print("No flow records found")

    except ImportError:
        print("Matplotlib not available - skipping visualization")
    except Exception as e:
        print(f"Error plotting hydrograph: {e}")
else:
    print("Skipping: results not available")

---

## Complete Workflow Summary

All phases of the AORC gridded precipitation workflow have been completed.

In [ ]:
print("=" * 70)
print("COMPLETE AORC GRIDDED PRECIPITATION WORKFLOW")
print("=" * 70)

print("\n[OK] Phase 1-2 (14a): AORC Download")
print("    HUC watersheds downloaded, storm catalog generated")
print("    AORC data downloaded and converted to DSS")

print("\n[OK] Phase 3 (14b): HMS Grid Configuration")
print("    Grid definition file created")
print("    Grid cell mapping (hrapcells) generated")

if TENK_AVAILABLE:
    print(f"\n[OK] Phase 4 (14c): HMS Execution")
    print(f"    Project: {hms_project.project_name}")
    if 'run_name' in dir():
        print(f"    Run: {run_name}")
    if 'element_peaks' in dir() and len(element_peaks) > 0:
        max_peak = max(max(p['peak_flow'] for p in peaks_list) 
                       for peaks_list in element_peaks.values())
        print(f"    Maximum peak flow: {max_peak:,.0f} cfs")
else:
    print("\n[SKIPPED] Phase 4 (14c): HMS Execution")
    print("    tenk project not available")

print(f"\nWorkflow completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## Next Steps: Using in Your Own HMS Project

To use AORC gridded precipitation in your own HEC-HMS project:

1. **Copy files to HMS project folder**:
   - `aorc_storm.dss` - Grid data
   - `aorc.grid` - Grid definition
   - `hrapcells` - Grid cell mapping

2. **Configure Meteorologic Model**:
   - Create new Met Model with type "Gridded Precipitation"
   - Select the grid definition
   - Associate subbasins with hrapcells regions

3. **Configure Basin Model**:
   - Create subbasins matching your HUC names
   - Set transform method to "ModClark" for distributed routing

4. **Run Simulation**:
   - Create control specification covering storm period
   - Create run combining basin, met, and control
   - Execute and analyze results

---

## Additional Resources

- **HmsHuc API**: `help(HmsHuc)` - HUC watershed operations
- **HmsAorc API**: `help(HmsAorc)` - AORC data operations
- **HmsGrid API**: `help(HmsGrid)` - Grid file operations
- **Example Project**: `tenk` in HMS examples has gridded precipitation
- **AORC Documentation**: https://registry.opendata.aws/noaa-nws-aorc/